In [2]:
from psmiles import PolymerSmiles as PS
import pandas as pd
# Read csv and filter EGC
egc_df = pd.read_csv("/home/ibrahim/recources/export.csv")
egc_df = egc_df[egc_df.property == "Egc"]

#creating fingerprints for SS
smile_string_list = list(egc_df.smiles.values)
value_list = list(egc_df.value.values)
finger_print_list = []
for ss in smile_string_list:
    curren_polymere = PS(ss)
    
    
    finger_print_list.append(curren_polymere.fingerprint())

summary_list = list(zip(smile_string_list,finger_print_list,value_list))
fp_list_formatted = []
for i in finger_print_list:
    fp_list_formatted.append(i.tolist())
print(len(fp_list_formatted[0]))
ss_fp_egc_df = pd.DataFrame({'Smile': smile_string_list, 'Finger_prints': fp_list_formatted, 'Egc': value_list})
print(ss_fp_egc_df)

2048
                                                 Smile  \
0                                              [*]C[*]   
1                                          [*]CC([*])C   
2                                         [*]CC([*])CC   
3                                        [*]CC([*])CCC   
4                                     [*]CC([*])CC(C)C   
...                                                ...   
3375           [*]Nc1c([2H])c([2H])c([*])c([2H])c1[2H]   
3376               [*]CCCCCC[N+](C)(C)CCC[N+]([*])(C)C   
3377          [*]CCCCCCCC[N+](C)(C)CCCCCC[N+]([*])(C)C   
3378  [*]CCCCCCCCCCCCCCCC[N+](C)(C)CCCCCC[N+]([*])(C)C   
3379       [*]C=Cc1cc([Si](C)(C)C)c([*])cc1[Si](C)(C)C   

                                          Finger_prints     Egc  
0     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  6.8972  
1     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  6.5196  
2     [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  6.5170  
3     [0, 1, 0, 0, 0, 0, 0, 0, 0, 

In [2]:
import numpy as np
import pandas as pd

final_df = pd.DataFrame(
    np.array(finger_print_list), index=smile_string_list
)

ss = pd.Series(value_list, index=smile_string_list, name="Egc")

concatenated_df = pd.concat(
    [final_df, ss],
    axis=1,
).reset_index(names="psmiles")
print(concatenated_df.set_index("psmiles").columns)

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
        2039,  2040,  2041,  2042,  2043,  2044,  2045,  2046,  2047, 'Egc'],
      dtype='object', length=2049)


In [3]:
import pandas as pd



auto_ml_df = concatenated_df.sample(frac=1, random_state=0)#shulles the df
print(auto_ml_df.shape)


psmlies_column = auto_ml_df["psmiles"]
auto_ml_df = auto_ml_df.drop(columns=["psmiles"])
auto_ml_df.reset_index(drop=True, inplace=True)
print(auto_ml_df.shape)
split_index = int(len(auto_ml_df) * 0.8)

train_df = auto_ml_df[:split_index]
test_df = auto_ml_df[split_index:]
# evtl min max scaler 

test_df.head()

(3380, 2050)
(3380, 2049)


,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,Egc
2704,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.0497
2705,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,5.6991
2706,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.9694
2707,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,5.7012
2708,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.3739


In [15]:
X_train = train_df.to_numpy()[:, :-1]
y_train = train_df.to_numpy()[:, -1]

X_test = test_df.to_numpy()[:, :-1] 
y_test = test_df.to_numpy()[:, -1]

(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)
print(X_train)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from autokeras import StructuredDataRegressor


ak = StructuredDataRegressor(max_trials=10,loss="mean_squared_error")

2023-09-25 13:52:23.531333: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 13:52:24.131148: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


In [14]:
ak.fit(x=X_train,y=y_train,epochs=1000,verbose=1)


Search: Running Trial #9

Value             |Best Value So Far |Hyperparameter
True              |True              |structured_data_block_1/normalize
False             |False             |structured_data_block_1/dense_block_1/use_batchnorm
2                 |2                 |structured_data_block_1/dense_block_1/num_layers
32                |128               |structured_data_block_1/dense_block_1/units_0
0.25              |0                 |structured_data_block_1/dense_block_1/dropout
1024              |32                |structured_data_block_1/dense_block_1/units_1
0                 |0                 |regression_head_1/dropout
adam              |adam              |optimizer
1e-05             |0.001             |learning_rate
128               |None              |structured_data_block_1/dense_block_1/units_2



KeyboardInterrupt: 

In [ ]:
model = ak.export_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2048)]            0         
                                                                 
 multi_category_encoding (M  (None, 2048)              0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 2048)              4097      
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                65568     
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

In [ ]:
import matplotlib.pyplot as plt

# Ihre vorhergesagten Werte für Trainings- und Testdaten
train_predictions = ak.predict(X_train)
test_predictions = ak.predict(X_test)

# Die echten Werte für Trainings- und Testdaten
real_train = y_train
real_test = y_test
"""
# Plotten der Daten
plt.scatter(train_predictions, real_train, label="Trainingsdaten", c="#d95f02", alpha=0.5)
plt.scatter(test_predictions, real_test, label="Testdaten", c="#7570b3", alpha=0.5)

plt.xlabel("Vorhergesagter Wert")
plt.ylabel("Echter Wert")
plt.legend()

plt.tight_layout()
plt.show()"""



85/85 [==============================] - 16s 191ms/step


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error , mean_squared_error

# Berechne R2 für Testdaten
r2_test = r2_score(real_test, test_predictions)
print("R2 für Testdaten:", r2_test)

# Berechne MAE für Testdaten
mae_test = mean_absolute_error(real_test, test_predictions)
print("MAE für Testdaten:", mae_test)

mse_test = mean_squared_error(real_test, test_predictions)
print("mse_test:", mse_test)

# Berechne R2 für Trainingsdaten
r2_train = r2_score(real_train, train_predictions)
print("R2 für Trainingsdaten:", r2_train)

# Berechne MAE für Trainingsdaten
mae_train = mean_absolute_error(real_train, train_predictions)
print("MAE für Trainingsdaten:", mae_train)

mse_train = mean_squared_error(real_train, train_predictions)
print("mse_train:", mse_train)

R2 für Testdaten: 0.4867586151862874
MAE für Testdaten: 0.7920412251729937
mse_test: 1.2184295541834265
R2 für Trainingsdaten: 0.9445100313213182
MAE für Trainingsdaten: 0.231219265700815
mse_train: 0.13648657608382353


In [ ]:

model.save("/home/ibrahim/models/SDRTEST")